Nested Data Models
================

```{important} Starting File: CHAPTER_5_WAFFLES
This chapter will start from the CHAPTER_5_WAFFLES and end on the CHAPTER_6_WAFFLES.
```

Data models are often more than flat objects. Many data structures and models can be precived as a series of nested dictionaries, or "models within models." We could validate those by hand, but *pydantic* provides the tools to handle that for us.

This chapter, we'll be covering nesting models within each other. We'll also be touching on a very powerfull tool for validating strings called Regular Expressions, or "regex."

```{admonition} Check Out Pydantic
:class: note
We will not be covering all the capabilities of *pydantic* here, and we highly encourage you to visit [the pydantic docs](https://pydantic-docs.helpmanual.io/) to learn about all the powerful and easy-to-execute things *pydantic* can do.
```

```{admonition} Compatibility with Python 3.8 and below
:class: note
If you have Python 3.8 or below, you will need to import container type objects such as `List`, `Tuple`, `Dict`, etc. from the `typing` library instead of their native types of `list`, `tuple`, `dict`, etc. This chapter will assume Python 3.9 or greater, however, both approaches will work in >=Python 3.9 and have 1:1 replacements of the same name.
```

## Nested Models: Just Dictionaries with Some Structure

Let's start by taking a look at our `Molecule` object once more and looking at some sample data.

In [1]:
import numpy as np
from pydantic import BaseModel, validator

class Molecule(BaseModel):
    name: str
    charge: float
    symbols: list[str]
    coordinates: np.ndarray
        
    class Config:
        arbitrary_types_allowed = True
    
    @validator("coordinates", pre=True)
    def coord_to_numpy(cls, coords):
        try:
            coords = np.asarray(coords)
        except ValueError:
            raise ValueError(f"Could not cast {coords} to numpy array")
        return coords
        
    @validator("coordinates")
    def coords_length_of_symbols(cls, coords, values):
        symbols = values["symbols"]
        if (len(coords.shape) != 2) or (len(symbols) != coords.shape[0]) or (coords.shape[1] != 3):
            raise ValueError(f"Coordinates must be of shape [Number Symbols, 3], was {coords.shape}")
        return coords
    
    @property
    def num_atoms(self):
        return len(self.symbols)

In [2]:
mol_data = {  # Good data
    "coordinates": [[0, 0, 0], [1, 1, 1], [2, 2, 2]], 
    "symbols": ["H", "H", "O"], 
    "charge": 0.0, 
    "name": "water"
}

bad_name = {"name": 789}  # Name is not str
bad_charge = {"charge": [1, 0.0]}  # Charge is not int or float
noniter_symbols = {"symbols": 1234567890}  # Symbols is an int
nonlist_symbols = {"symbols": '["H", "H", "O"]'}  # Symbols is a string (notably is a string-ified list)
tuple_symbols = {"symbols": ("H", "H", "O")}  # Symbols as a tuple?
bad_coords = {"coordinates": ["1", "2", "3"]}  # Coords is a single list of string
inner_coords_not3d = {"coordinates": [[1, 2, 3], [4, 5]]}
bad_symbols_and_cords = {"symbols": ["H", "H", "O"],
                         "coordinates": [[1, 1, 1], [2.0, 2.0, 2.0]]
                        }  # Coordinates top-level list is not the same length as symbols

What exactly is our model? Our model is a `dict` with specific keys `name`, `charge`, `symbols`, and `coordinates`; all of which have some restrictions in the form of type annotations. What if we had another model for additional information that needed to be kept together, and those data do not make sense to transfer to a flat list of other attributes? An example of this would be contributor-like metadata; the origonator or provider of the data in question.

Let's make one up. Say the information follows these rules: 
* A name of the contributor
* A contact email or homepage
* An optional organization.
* The contributor as a whole is optional too.

In [14]:
contributor = {"name": "I. Taylor Researcher",
               "url": "https://molssi.org",
               "Organization": "The Molecular Sciences Software Institute"
              }

That looks like a good contributor of our `mol_data`. How would we add this entry to the `Molecule`? First let's understand what an "optional" entry is.

### Defining Optional Entries

For type hints/annotations, "optional" translates to "default `None`". This can be specified in one of two main ways, three if you are on Python 3.10 or greater.

In [7]:
from typing import Optional, Any

contributor: Optional[Any]
contributor: Any = None
contributor: Any | None  # Python 3.10+ only

First thing to note is the `Any` object from `typing`. This can be used to mean exactly that, *any* data are valid here. We'll replace it with our actual model in a moment. Let's go over the wys to specify optional entries now with the understanding that all three of these mean and do the *exact* same thing.

`Optional[Any]` borrows the `Optional` object from the `typing` library. 

`Any = None` sets a default value of `None`, which also implies optional. This method can be used in tandem with any other type and not `None` to set a default value. E.g. `variable: int = 12` would indicate an `int` type hint, and default value of `12` if its not set in the input data.

`Any | None` employs the set operators with Python to treat this as "any OR none". This is also equal to `Union[Any,None]`. This only works in Python 3.10 or greater and it should be noted this will be the prefered way to specify `Union` in the future, removing the need to import it at all.

### Understanding the `dict` Type

At the end of the day, all models are just glorified dictionaries with conditions on what is and is not allowed. You can specify a `dict` type which up to 2 arguments for its type hints: keys and values, in that order. Although the Python dictionary supports any immutable type for a dictionary key, *pydantic* models accept only strings by default (this can be changed). 

If we take our `contributor` rules, we could define this sub model like such:

In [9]:
class ValidEmail(str):
    pass

class ValidHTML(str):
    pass

from typing import Union
contributor: dict[
    str,  # Key Type Annotation
    Union[  # Value Type Annotation
        ValidEmail, ValidHTML, str  # Different accepted string types, overly permissive
    ],  
] = None  # Optional

We would need to fill in the rest of the validator data for `ValidURL` and `ValidHTML`, write some rather rigourus validation to ensure there are only the correct keys, and ensure the values all adhere to the other rules above, but it can be done. In fact, the values `Union` is overly permissive.


<div class="exercise">
<p class="exercise-title"> Why is the values <code>Union</code> overly permisive?</p>
<p>As written, the <code>Union</code> will not actually correctly prevent bad URLs or bad emails, why?</p>

```{admonition} Solution:
:class: dropdown
Because *pydantic* runs its validators in order until one succeeds or all fail, any string will correctly validate once it hits the <code>str</code> type annotation at the very end. 

</div>

## Nested Models: The Pydantic Way

Manually writting validators for structured models within our models made simple with *pydantic*. Because our `contributor` is just another model, we can treat it as such, and inject it in any other *pydantic* model.

In [11]:
from pydantic import BaseModel, validator

class Contributor(BaseModel):
    name: str
    url: str
    Organization: str = None  # Set default that makes it optional

There it is, our very basic model. Because this is just another *pydantic* model, we can also write validators that will run for just this model. We'll revist that concept in a moment though, and lets inject this model into our exiting *pydantic* model for `Molecule`.

In [13]:
import numpy as np
from pydantic import BaseModel, validator

class Molecule(BaseModel):
    name: str
    charge: float
    symbols: list[str]
    coordinates: np.ndarray
    contributor: Contributor = None # <--- New, nested, optional model
        
    class Config:
        arbitrary_types_allowed = True
    
    @validator("coordinates", pre=True)
    def coord_to_numpy(cls, coords):
        try:
            coords = np.asarray(coords)
        except ValueError:
            raise ValueError(f"Could not cast {coords} to numpy array")
        return coords
        
    @validator("coordinates")
    def coords_length_of_symbols(cls, coords, values):
        symbols = values["symbols"]
        if (len(coords.shape) != 2) or (len(symbols) != coords.shape[0]) or (coords.shape[1] != 3):
            raise ValueError(f"Coordinates must be of shape [Number Symbols, 3], was {coords.shape}")
        return coords
    
    @property
    def num_atoms(self):
        return len(self.symbols)

That one line has now added the entire construct of the `Contributor` model to the `Molecule`. The name of the submodel does NOT have to match the name of the attribute its representing. *Pydantic* will handle passing off the nested dictionary of input data to the nested model and construct it according to its own rules.

In [16]:
water = Molecule(**mol_data)
print(water)

name='water' charge=0.0 symbols=['H', 'H', 'O'] coordinates=array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2]]) contributor=None


In [19]:
mol_data_with_contrib = {**mol_data, **{"contributor": contributor}}
water_with_contributor = Molecule(**mol_data_with_contrib)
print(water_with_contributor)

name='water' charge=0.0 symbols=['H', 'H', 'O'] coordinates=array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2]]) contributor=Contributor(name='I. Taylor Researcher', url='https://molssi.org', Organization='The Molecular Sciences Software Institute')


And that's the basic of nested models. Aribtrary levels of nesting and peicewise addition of models can be constructed and inherted to make rich data structures.

We still have the mater of making sure the URL is a valid url or email link, and for that we'll need to touch on Regular Expressions.

## Validating Strings on Patterns: Regular Expressions

Strings, all strings, have patterns in them. Those patterns can be descibed with a specialized pattern recognition language called [Regular Expressions or "regex"](https://en.wikipedia.org/wiki/Regular_expression). A full understanding of regex is NOT required nor expected for this workshop. However, we feel its important to touch on as the more data validation you do, especially on strings, the more likely it will be that you need regex at some point.

Let's write a validator for email. We're looking for something that looks like `mailto:someemail@fake-location.org`. And maybe the `mailto:` part is optional. We start by creating our validator by subclassing `str`.

In [22]:
import re

class MailTo(str):
    @classmethod
    def __get_validators__(cls):
        for validator in [cls.valid_str, cls.valid_email]:
            yield validator

    @classmethod
    def valid_str(cls, v: str):
        if not isinstance(v, str):
            raise TypeError("Not a valid mailto Email format")
        v = v.strip()
        return v
    
    @classmethod
    def valid_email(cls, v: str):
        if not re.match(r"(mailto:)?[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]+", v):
            raise ValueError("mailto URL is not a valid mailto or email link")
        return v

This is the custom validator form of the supplementary material in the last chapter REF_THIS_WAFFLES. If you did not go through that section, don't worry. The short of it is this is the form for making a custom type and providing built-in validation methods for *pydantic* to access. Each of the `valid_X` functions have ben set up to run as different things which have to be validated for something of typr `MailTo` to be considered valid. The important part to focus on here is the `valid_email` function and the `re.match` method.

`re` is a built-in Python library for doing regex. The `match(pattern, string_to_find_match)` function looks for the `pattern` from the first character of `string_to_find_match`. Our pattern can be broken down into the following way:

```
r"              <-- Literal String, bypasses alot of special characters like #
(               <-- Grouping
    mailto:     <-- exactly this, no special characters here
)?              <-- ? Optional, operating on Grouping
[               <-- Single character matching any of the characters within
    a-z         <-- any lowercase a to z character
    A-Z         <-- any uppercase A to Z character
    0-9         <-- any number 0 to 9
    ._%+-       <-- any of these characters, special regex meaning supressed inside []
]+              <-- + at least one instance, repeated as many times, operating on single character match = "these characters at least once"
@               <-- literal @ sign 
[a-zA-Z0-9.-]+  <-- Single character match, of this pattern, at least one of
\.              <-- literal "." because \ supresses wildcard nature of . in regex
[a-zA-Z]+       <-- Single character match, of this pattern, at least one of
"               <-- End pattern

```

We're not expecting this to be memoriezed, just to understand that there is a pattern that is being looked for. We can now set this pattern as one of the valid paremters of the `url` entry in the `contributor` model. 

In [23]:
from typing import Union

from pydantic import BaseModel, validator

class Contributor(BaseModel):
    name: str
    url: Union[MailTo, str]
    Organization: str = None  # Set default that makes it optional

So why did we show this if we were only going to pass in `str` as the second `Union` option? We wanted to show this regex pattern as *pydantic* provides a number of helper "types" which function very similar to our custom `MailTo` class that can be used to shortcut writting manual validators. Their names often say exactly what they do. Some examples include:

* StrictInt
* PositiveFloat
* AnyUrl
* stricturl
* IPvAnyAddress
* and many more

They also have constrained types which you can use to set some boundaries without having to code them yourself. Natively, we can use the `AnyUrl` to save us having to write our own regex validator for matching URLs.

In [24]:
from typing import Union

from pydantic import BaseModel, validator, AnyUrl

class Contributor(BaseModel):
    name: str
    url: Union[MailTo, AnyUrl]
    Organization: str = None  # Set default that makes it optional

<div class="exercise">
<p class="exercise-title"> Challenge: URL Regex Validator
    <p>Write a custom match string for a URL regex pattern</p>

```{admonition} Do not do this yourself
:class: warning
There are many correct answers. All of them are extremly difficult regex strings. Put some thought into your answer, understanding that its best to look up an answer (feel free to do this), or borrow from someone else; with attribution. We did this for this challenge as well.
```
    
```{admonition} Answer:
:class: dropdown
With credit: https://gist.github.com/gruber/8891611#file-liberal-regex-pattern-for-web-urls-L8
```python
r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
```

</div>

## Final Code: Bringing it All Together.

Let's combine everything we've built into one final block of code

In [26]:
import re
from typing import Union

import numpy as np
from pydantic import BaseModel, validator, AnyUrl


class MailTo(str):
    @classmethod
    def __get_validators__(cls):
        for validator in [cls.valid_str, cls.valid_email]:
            yield validator

    @classmethod
    def valid_str(cls, v: str):
        if not isinstance(v, str):
            raise TypeError("Not a valid mailto Email format")
        v = v.strip()
        return v
    
    @classmethod
    def valid_email(cls, v: str):
        if not re.match(r"(mailto:)?[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]+", v):
            raise ValueError("mailto URL is not a valid mailto or email link")
        return v

    
class Contributor(BaseModel):
    name: str
    url: Union[MailTo, AnyUrl]
    Organization: str = None  # Set default that makes it optional


class Molecule(BaseModel):
    name: str
    charge: float
    symbols: list[str]
    coordinates: np.ndarray
    contributor: Contributor = None
        
    class Config:
        arbitrary_types_allowed = True
    
    @validator("coordinates", pre=True)
    def coord_to_numpy(cls, coords):
        try:
            coords = np.asarray(coords)
        except ValueError:
            raise ValueError(f"Could not cast {coords} to numpy array")
        return coords
        
    @validator("coordinates")
    def coords_length_of_symbols(cls, coords, values):
        symbols = values["symbols"]
        if (len(coords.shape) != 2) or (len(symbols) != coords.shape[0]) or (coords.shape[1] != 3):
            raise ValueError(f"Coordinates must be of shape [Number Symbols, 3], was {coords.shape}")
        return coords
    
    @property
    def num_atoms(self):
        return len(self.symbols)

Congratulations! You've now written a robust data model with automatic type annotations, validation, and complex structure including nested models.

Our `Molecule` has come a long way from being a simple data class with no validation. We learned how to annotate the arguments with built-in Python type hints. We converted our data structure to a Python `dataclass` to simplify repettitive code and make our structure easier to understand. The data were validated through manual checks which we learned could be programmatically handled. *Pydantic* was brought in to turn our type hints into type annotations and automatically check typing, both Python-native and external/custom types like NumPy arrays. Finally we created nested models to permit arbitrary complexity and a better understanding of what tools are available for validating data.

Finally, we encourage you to go through and visit all the external links in these chapters, especially for *pydantic*. This workshop only touched on basic *pydantic* usage, and there is so much more you can do with auto-validating models. 

We hope you've found this workshop helpful and we welcome any comments, feedback, spotted issues, improvements, or suggestions on the material through the GitHub (link as a dropdown at the top.)